#### Wrangle Spark Exercises

These exercises should go in a notebook or script named `wrangle`. Add, commit, and push your changes.

This exercises uses the `case.csv`, `dept.csv`, and `source.csv` files from the san antonio 311 call dataset.

1. Read the case, department, and source data into their own spark dataframes.

2. Let's see how writing to the local disk works in spark:

 - Write the code necessary to store the source data in both csv and json format, store these as `sources_csv` and `sources_json`.
 - Inspect your folder structure. What do you notice?


3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [3]:
# 1. read exercise dataframes into spark dataframes
case_df = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)
dept_df = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)
source_df = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)

In [4]:
# write to local disk in scv and json

case_df.write.json("case.df_json", mode = "overwrite")
case_df.write.csv("case.df_csv", mode = "overwrite")

dept_df.write.json("dept.df_json", mode = "overwrite")
dept_df.write.csv("dept.df_csv", mode = "overwrite")

source_df.write.json("source.df_json", mode = "overwrite")
source_df.write.csv("source.df_csv", mode = "overwrite")

# spark proceeds to create folders for each of these dataframes for their respective types. 
# furthermore, files are split into almost equally sized files instead of one consolidated
# file.

In [5]:
# inspect data in dataframes
# case

case_df.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [6]:
# inspect data in dataframes
# dept

dept_df.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

In [7]:
# inspect data in dataframes
# source

source_df.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [8]:
# Rename column
case_df = case_df.withColumnRenamed('SLA_due_date', 'case_due_date')

# Convert to better data types
case_df = (case_df.withColumn('case_late', col('case_late') == 'YES')
    .withColumn('case_closed', col('case_closed') == 'YES')
)
case_df = case_df.withColumn('council_district', format_string('%04d', col('council_district')))
case_df = (case_df.withColumn('case_opened_date', to_timestamp(col('case_opened_date'), 'M/d/yy H:mm'))
    .withColumn('case_closed_date', to_timestamp(col('case_closed_date'), 'M/d/yy H:mm'))
    .withColumn('case_due_date', to_timestamp(col('case_due_date'), 'M/d/yy H:mm'))
)

# Cleanup text data
case_df = case_df.withColumn('request_address', lower(trim(col('request_address'))))
# Extract zipcode
case_df = case_df.withColumn('zipcode', regexp_extract(col('request_address'), r'\d+$', 0))

# Create a `case_lifetime` feature
case_df = (
    case_df.withColumn('case_age', datediff(current_timestamp(), 'case_opened_date'))
    .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
    .withColumn('case_lifetime', when(col('case_closed'), col('days_to_closed')).otherwise(col('case_age')))
    .drop('case_age', 'days_to_closed')
)

# Join departments and sources
df = case_df.join(dept_df, 'dept_division', 'left').join(source_df, 'source_id', 'left')

1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [9]:
# How old is the latest (in terms of days past SLA) currently open issue?
df.show(1, vertical = True)

-RECORD 0--------------------------------------
 source_id              | svcCRMLS             
 dept_division          | Field Operations     
 case_id                | 1014127332           
 case_opened_date       | 2018-01-01 00:42:00  
 case_closed_date       | 2018-01-01 12:29:00  
 case_due_date          | 2020-09-26 00:42:00  
 case_late              | false                
 num_days_late          | -998.5087616000001   
 case_closed            | true                 
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 request_address        | 2315  el paso st,... 
 council_district       | 0005                 
 zipcode                | 78207                
 case_lifetime          | 0                    
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | YES                  
 source_username        | svcCRMLS      

In [14]:
(df.select(datediff(current_timestamp(), 'case_due_date')
 .alias('days_past_due'))
 .where(df.case_closed == False)
 .sort(col('days_past_due').desc())
 .show(5)
)

+-------------+
|days_past_due|
+-------------+
|         1856|
|         1856|
|         1856|
|         1855|
|         1853|
+-------------+
only showing top 5 rows



In [24]:
#How long has the oldest (in terms of days since opened) currently opened issue been open?
df.createOrReplaceTempView('df')

spark.sql("""
SELECT DATEDIFF(current_timestamp, case_opened_date) AS days_past_opened
FROM df
WHERE NOT case_closed
ORDER BY days_past_opened DESC
LIMIT 5
""").show()

+----------------+
|days_past_opened|
+----------------+
|            1872|
|            1872|
|            1871|
|            1870|
|            1869|
+----------------+



2. How many Stray Animal cases are there?

In [25]:
(df.groupBy('service_request_type')
 .count()
 .filter(expr('service_request_type == "Stray Animal"'))
 .show()
)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|        Stray Animal|27361|
+--------------------+-----+



3. How many service requests that are assigned to the Field Operations department (`dept_division`) are not classified as "Officer Standby" request type (`service_request_type`)?

In [26]:
(df.filter(df.dept_division == 'Field Operations')
 .filter(df.service_request_type != 'Officer Standby')
 .count()
)

116295

4. Convert the `council_district` column to a string column.

In [20]:
df = df.withColumn('council_district', col('council_district').cast('string'))
df.dtypes

[('source_id', 'string'),
 ('dept_division', 'string'),
 ('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('zipcode', 'string'),
 ('case_lifetime', 'int'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string'),
 ('source_username', 'string')]

5. Extract the year from the `case_closed_date` column.

In [28]:
(df.select('case_closed_date', year('case_closed_date').alias('year')).show(5)
)

+-------------------+----+
|   case_closed_date|year|
+-------------------+----+
|2018-01-01 12:29:00|2018|
|2018-01-03 08:11:00|2018|
|2018-01-02 07:57:00|2018|
|2018-01-02 08:13:00|2018|
|2018-01-01 13:29:00|2018|
+-------------------+----+
only showing top 5 rows



6. Convert `num_days_late` from days to hours in new columns num_hours_late.

In [33]:
(
    df.withColumn('num_hours_late', df.num_days_late * 24)
    .select(round('num_days_late', 2)
            .alias('num_days_late'), round('num_hours_late', 2)
            .alias('num_hours_late'))
    .show(5)
)

+-------------+--------------+
|num_days_late|num_hours_late|
+-------------+--------------+
|      -998.51|     -23964.21|
|        -2.01|         -48.3|
|        -3.02|        -72.54|
|       -15.01|       -360.28|
|         0.37|          8.93|
+-------------+--------------+
only showing top 5 rows



7. Join the case data with the source and department data.

In [ ]:
#df = case_df.join(dept_df, 'dept_division', 'left').join(source_df, 'source_id', 'left')

8. Are there any cases that do not have a request source?

9. What are the top 10 service request types in terms of number of requests?

10. What are the top 10 service request types in terms of average days late?

11. Does number of days late depend on department?

12. How do number of days late depend on department and request type?